# 🚀 Day 3-4: 앙상블 심화 - 스태킹(Stacking)과 블렌딩(Blending)

이전 세션까지 우리는 여러 개별 모델의 장단점을 배우고, 데이터를 효과적으로 처리하는 `Pipeline`을 구축하는 방법을 익혔습니다. 

하지만 단일 모델만으로는 예측 성능의 한계에 부딪힐 때가 많습니다. 이럴 때 필요한 것이 바로 **앙상블(Ensemble)** 기법입니다.

앙상블은 여러 개의 '약한 학습기(Weak Learner)'를 결합하여 하나의 '강한 학습기(Strong Learner)'를 만드는 방법론입니다. 

우리는 이미 Bagging(RandomForest)과 Boosting(XGBoost, LightGBM)이라는 대표적인 앙상블 기법을 다루어 보았습니다.

이번 파트에서는 앙상블의 '끝판왕'이라 불리는 **스태킹(Stacking)** 과 그 변형인 **블렌딩(Blending)** 에 대해 심도 있게 알아봅니다.

 이 두 기법은 여러 다른 종류의 모델 예측 결과를 다시 한번 학습시켜, 개별 모델의 장점은 취하고 단점은 보완하는 매우 강력한 전략입니다. 
 
 특히 캐글(Kaggle)과 같은 데이터 분석 경진대회에서 우승을 노리는 참가자들이 즐겨 사용하는 비장의 무기이기도 합니다.

### 학습 목표
1.  **스태킹(Stacking)** 의 개념과 동작 원리를 이해하고, 데이터 누수(Data Leakage)를 방지하는 방법을 학습합니다.
2.  `scikit-learn`의 `StackingRegressor`를 사용하여 스태킹 모델을 구현합니다.
3.  **블렌딩(Blending)** 의 개념과 스태킹과의 차이점을 이해합니다.
4.  훈련 데이터를 분할하여 블렌딩 모델을 직접 구현합니다.
5.  개별 모델, 스태킹, 블렌딩의 성능을 비교 분석하고 장단점을 토론합니다.

---

### 1. 실습 준비: 라이브러리 및 데이터 로딩

이번 실습에서는 미국 아이오와 주 에임스(Ames)시의 주택 가격 예측 데이터셋을 사용하겠습니다. 다양한 특성을 가지고 있어 회귀 모델의 성능을 비교하기에 좋은 데이터입니다.

먼저 필요한 라이브러리를 임포트하고 데이터를 준비하겠습니다.

#### 💻 코드로 알아보기: 데이터 준비

In [3]:

import pandas as pd
import numpy as np
import plotly.express as px

# 모델 및 전처리 도구
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# 앙상블 모델
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

* 데이터셋 설명
  
    CSV 파일: 19,237행 x 18열 (가격 컬럼이 타겟 변수로 포함)
  
    | 속성 | 설명 |
    |------|------|
    | ID | 식별자 |
    | Price | 차량 가격 (타겟 변수) |
    | Levy | 세금 |
    | Manufacturer | 제조사 |
    | Model | 모델명 |
    | Prod. year | 생산년도 |
    | Category | 차량 종류 |
    | Leather interior | 가죽 시트 여부 |
    | Fuel type | 연료 종류 |
    | Engine volume | 엔진 배기량 |
    | Mileage | 주행거리 |
    | Cylinders | 실린더 수 |
    | Gear box type | 변속기 종류 |
    | Drive wheels | 구동 방식 |
    | Doors | 문 개수 |
    | Wheel | 휠 위치 |
    | Color | 색상 |
    | Airbags | 에어백 개수 |

In [9]:
# 데이터 로드
path = "../datasets/ml/car-price-prediction/car_price_prediction.csv"
df = pd.read_csv(path)

In [10]:
df.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


In [11]:
# 데이터 타입 변환 및 결측치 처리
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df.dropna(subset=['Price'], inplace=True)

# 특성(X)과 타겟(y) 분리
X = df.drop('Price', axis=1)
y = df['Price']

# 훈련 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 전처리 파이프라인 구축
# 수치형과 범주형 특성 구분
numeric_features = X.select_dtypes(include=np.number).columns
categorical_features = X.select_dtypes(include='object').columns

# 수치형 데이터 전처리: 결측치를 중앙값으로 채우고 표준화 스케일링
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 범주형 데이터 전처리: 결측치를 'missing'으로 채우고 원-핫 인코딩
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# ColumnTransformer를 사용하여 각 특성에 맞는 전처리 적용
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

print("데이터 준비 완료!")
print(f"훈련 데이터 크기: {X_train.shape}")
print(f"테스트 데이터 크기: {X_test.shape}")

데이터 준비 완료!
훈련 데이터 크기: (15389, 17)
테스트 데이터 크기: (3848, 17)


### 2. 베이스라인 모델 성능 확인

스태킹과 블렌딩의 효과를 확인하기 위해, 먼저 개별 모델(Base Model)들의 성능을 측정해보겠습니다. 이를 **베이스라인(Baseline)** 이라고 합니다. 서로 다른 유형의 모델을 사용하는 것이 앙상블 효과를 극대화하는 데 도움이 됩니다. 

  - **Lasso**: 규제가 있는 선형 모델
  - **RandomForestRegressor**: Bagging 기반의 트리 앙상블 모델
  - **XGBRegressor**: Boosting 기반의 트리 앙상블 모델 

각 모델을 파이프라인에 연결하여 학습하고, **RMSE(Root Mean Squared Error)**로 성능을 평가합니다.

#### 💻 코드로 알아보기: 베이스라인 모델 학습 및 평가

In [12]:
# 모델 정의
lasso = Lasso(random_state=42)
rf = RandomForestRegressor(n_estimators=100, random_state=42)
xgb = XGBRegressor(random_state=42)

# 모델별 파이프라인 생성 및 평가
models = {'Lasso': lasso, 'RandomForest': rf, 'XGBoost': xgb}
baseline_results = {}

for name, model in models.items():
    # 전처리기와 모델을 파이프라인으로 연결
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', model)])

    # 모델 학습
    pipeline.fit(X_train, y_train)

    # 예측 및 RMSE 계산
    y_pred = pipeline.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    baseline_results[name] = rmse
    print(f"{name} 모델의 RMSE: {rmse:.4f}")

Lasso 모델의 RMSE: 85183.2413
RandomForest 모델의 RMSE: 9953.2208
XGBoost 모델의 RMSE: 15923.4639


이제 각 모델의 기본 성능을 확인했습니다. 이 값들을 기준으로 스태킹과 블렌딩이 얼마나 성능을 향상시키는지 살펴보겠습니다.

-----

### 3\. 스태킹 (Stacking)

#### 🧠 개념 이해하기: 스태킹이란?

스태킹(Stacking, 또는 Stacked Generalization)은 여러 다른 모델(기본 모델)의 예측값을 새로운 특성(feature)으로 사용하여, 최종 예측을 하는 또 다른 모델(메타 모델)을 학습시키는 기법입니다. 

마치 **여러 분야의 전문가(기본 모델)들에게 자문을 구한 뒤, 그 의견들을 종합하여 최종 결정을 내리는 팀장(메타 모델)과 같습니다.**

  - **1단계 (Level-0)**: 여러 개의 **기본 모델(Base Models)** 들이 원본 훈련 데이터로 학습합니다. 서로 다른 종류의 모델을 사용하는 것이 일반적입니다. (e.g., 선형 모델, 트리 모델, 신경망 등)
  - **2단계 (Level-1)**: 1단계에서 학습된 기본 모델들이 예측한 결과를 입력 데이터로 사용하여 **메타 모델(Meta-Model)** 을 학습시킵니다.

**핵심 포인트: 데이터 누수(Data Leakage)와 Out-of-Fold (OOF) 예측**

스태킹을 구현할 때 가장 주의해야 할 점은 **데이터 누수**입니다. 만약 기본 모델을 훈련시킨 데이터와 동일한 데이터로 예측값을 만들어 메타 모델을 훈련시킨다면, 메타 모델은 이미 정답을 알고 있는 기본 모델의 예측에 과적합(overfitting)될 위험이 매우 큽니다.

이 문제를 해결하기 위해 **K-Fold 교차 검증을 활용한 Out-of-Fold (OOF) 예측** 방식을 사용합니다. 과정은 다음과 같습니다.

1.  훈련 데이터를 K개의 Fold(그룹)로 나눕니다. (예: 5-Fold)
   
2.  첫 번째 Fold를 검증(Validation) 데이터로, 나머지 K-1개 Fold를 훈련 데이터로 사용합니다.
3.  K-1개 Fold로 기본 모델들을 학습시킨 후, 이 모델들로 첫 번째 Fold(검증 데이터)에 대한 예측을 수행합니다. 이 예측값이 첫 번째 Fold에 대한 OOF 예측값이 됩니다.
4.  이 과정을 K번 반복하여 모든 Fold가 한 번씩 검증 데이터로 사용되도록 합니다.
5.  결과적으로, 전체 훈련 데이터에 대해 "해당 데이터가 학습에 사용되지 않은" 모델이 만든 예측값을 얻게 됩니다. 이것이 바로 메타 모델을 학습시킬 새로운 특성(`meta-feature`)이 됩니다.
6.  테스트 데이터에 대한 예측은, 각 Fold에서 학습된 K개의 기본 모델들이 테스트 데이터에 대해 예측한 값들의 평균을 사용합니다.

  <img src="https://miro.medium.com/v2/resize:fit:1400/0*l2nIa-PxTvHNtwm-.jpg" width="1000">

Scikit-learn의 `StackingRegressor`는 이 복잡한 과정을 `cv` 파라미터 하나로 간단하게 처리해줍니다.

#### 💻 코드로 알아보기: `StackingRegressor` 활용

이제 베이스라인에서 사용했던 모델들을 기본 모델로, 그리고 새로운 `Ridge` 모델을 메타 모델로 사용하여 스태킹 앙상블을 구축해 보겠습니다.

In [ ]:
# 1. 기본 모델(Level-0)과 메타 모델(Level-1) 정의
base_estimators = [
    ('random_forest', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('xgboost', XGBRegressor(random_state=42))
]

meta_estimator = Ridge(random_state=42)

# 2. 전처리 파이프라인과 StackingRegressor 결합
# StackingRegressor 자체에 estimators와 final_estimator를 지정합니다.
# StackingRegressor는 내부적으로 각 estimator를 fit할 때 전체 X_train을 사용합니다.
# 따라서 estimator를 파이프라인으로 감싸서 전처리가 포함되도록 해야 합니다.

# 각 기본 모델을 위한 파이프라인 생성
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', base_estimators[0][1])])
xgb_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', base_estimators[1][1])])

# Stacking에 사용할 (이름, 파이프라인) 리스트
stacking_estimators = [
    ('random_forest', rf_pipeline),
    ('xgboost', xgb_pipeline)
]

# 3. StackingRegressor 생성 및 학습
# final_estimator도 전처리가 필요할 수 있으므로 파이프라인으로 만듭니다.
# 여기서는 메타 모델이 선형 모델이므로 스케일링을 적용합니다.
meta_pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('ridge', meta_estimator)])

# StackingRegressor 객체 생성
# [cite_start]cv=5 : 5-Fold 교차 검증으로 OOF 예측값을 생성하여 메타 모델을 학습시킵니다. 
stacking_regressor = StackingRegressor(
    estimators=stacking_estimators,
    final_estimator=meta_pipeline,
    cv=5,
    n_jobs=-1
)

# 스태킹 모델 학습
stacking_regressor.fit(X_train, y_train)

# 예측 및 성능 평가
y_pred_stack = stacking_regressor.predict(X_test)
rmse_stack = np.sqrt(mean_squared_error(y_test, y_pred_stack))

print(f"스태킹 앙상블의 RMSE: {rmse_stack:.4f}")

# 결과 비교를 위해 저장
baseline_results['Stacking'] = rmse_stack

스태킹 모델의 성능이 개별 모델들보다 향상되었는지 확인해보세요. 서로 다른 모델의 예측을 조합함으로써 개별 모델이 가진 단점을 보완하고, 더 일반화된 예측을 만들어 낼 수 있습니다. 

#### ✏️ 연습문제 1: 스태킹 모델 변경하기

  - **문제**: 위 `StackingRegressor`의 `final_estimator`(메타 모델)를 `Lasso(alpha=1.0)`로 변경하고, 기본 모델(`estimators`)에 `GradientBoostingRegressor(n_estimators=50, random_state=42)`를 추가하여 새로운 스태킹 모델을 만들어보세요.
  - **목표**: 변경된 모델을 학습시키고 테스트 데이터에 대한 RMSE를 계산하여 이전 스태킹 모델 및 베이스라인 모델들과 성능을 비교해보세요.

-----

### 4. 블렌딩 (Blending)

#### 🧠 개념 이해하기: 블렌딩이란?

블렌딩은 스태킹의 간단한 버전이라고 생각할 수 있습니다.  

스태킹이 K-Fold 교차 검증으로 메타 모델 학습 데이터를 만드는 반면, 블렌딩은 훈련 데이터의 일부를 따로 떼어내 **검증 세트(Validation set 또는 Hold-out set)**로 만들고, 

이 세트를 이용해서만 메타 모델을 학습시킵니다. 

이 과정을 "시험 준비"에 비유해볼 수 있습니다. 

1.  **기본 모델 학습 (문제집 풀이)**: 전체 훈련 세트($$X_{train}$$)를 다시 훈련용(`train_a`, 예: 80%)과 검증용(`valid_b`, 예: 20%)으로 나눕니다.
   
2.  여러 기본 모델(학생들)은 `train_a`(문제집)만 보고 열심히 공부(학습)합니다.
3.  **메타 데이터 생성 (모의고사)**: 공부를 마친 기본 모델들에게 한 번도 본 적 없는 `valid_b`(모의고사)를 풀게 합니다. 이때 각 모델이 제출한 답안지(예측값)들이 바로 메타 모델이 학습할 데이터가 됩니다.
4.  **메타 모델 학습 (오답노트 분석)**: 메타 모델(선생님)은 학생들이 모의고사에서 어떤 문제를 어떻게 틀리고 맞혔는지(`valid_b`에 대한 예측값과 실제 정답)를 보고, 어떤 학생의 의견을 더 신뢰할지, 어떤 상황에서 특정 학생이 실수를 하는지 등의 패턴을 학습합니다. 
5.  **최종 예측 (실제 시험)**: 실제 시험(`X_test`)에서는, 기본 모델들이 각자 예측을 내놓으면, 오답노트 분석을 마친 메타 모델이 그 예측들을 종합하여 최종 정답을 제출합니다.

이 방식은 K-Fold를 사용하지 않아 구현이 간단하고 빠르지만, 훈련 데이터의 일부(`valid_b`)를 기본 모델 학습에 사용하지 못하는 단점이 있습니다. 

  <img src="https://blog.kakaocdn.net/dn/XhWPG/btsJcRZwO6c/d7LGfZP4CnWJTzfv0KqtJ0/img.png">

#### 💻 코드로 알아보기: 블렌딩 직접 구현하기

Scikit-learn에는 `BlendingRegressor`가 따로 없으므로, 위 개념에 따라 직접 구현해보겠습니다. 

In [ ]:
# 1. 훈련 데이터를 다시 훈련(train_a)과 검증(valid_b) 세트로 분할
# train_test_split을 한 번 더 사용하여 hold-out 세트를 만듭니다.
X_train_a, X_valid_b, y_train_a, y_valid_b = train_test_split(
    X_train, y_train, test_size=0.3, random_state=42
)

print(f"기본 모델 훈련 데이터 크기: {X_train_a.shape[0]}")
print(f"메타 모델 훈련 데이터 크기: {X_valid_b.shape[0]}")

# 2. 기본 모델들을 전처리 후 train_a 데이터로 학습
base_models = [
    ('lasso', Lasso(random_state=42)),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('xgb', XGBRegressor(random_state=42))
]

# 학습된 모델들을 저장할 딕셔너리
trained_base_models = {}

for name, model in base_models:
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', model)])
    pipeline.fit(X_train_a, y_train_a)
    trained_base_models[name] = pipeline
    print(f"{name} 모델 학습 완료.")

# 3. 메타 모델의 학습 데이터(meta_features) 생성
# 학습된 기본 모델들로 검증 세트(valid_b)를 예측합니다.
meta_features_valid = pd.DataFrame()
for name, model in trained_base_models.items():
    # valid_b 데이터에 대한 예측값을 새로운 특성으로 추가
    meta_features_valid[name] = model.predict(X_valid_b)

# 4. 메타 모델 학습
# 메타 모델은 valid_b에 대한 예측값과 실제 정답(y_valid_b)으로 학습합니다.
meta_model_blend = Ridge(random_state=42)
meta_model_blend.fit(meta_features_valid, y_valid_b)
print("\n메타 모델 학습 완료.")

# 5. 최종 예측 수행
# (1) 기본 모델들로 테스트 데이터(X_test)에 대한 예측 생성
meta_features_test = pd.DataFrame()
for name, model in trained_base_models.items():
    meta_features_test[name] = model.predict(X_test)

# (2) 생성된 메타 특성을 메타 모델에 입력하여 최종 예측
y_pred_blend = meta_model_blend.predict(meta_features_test)
rmse_blend = np.sqrt(mean_squared_error(y_test, y_pred_blend))

print(f"\n블렌딩 앙상블의 RMSE: {rmse_blend:.4f}")

# 결과 비교를 위해 저장
baseline_results['Blending'] = rmse_blend

블렌딩의 성능은 hold-out 세트를 어떻게 나누느냐에 따라 달라질 수 있습니다.  스태킹보다 구현은 간단하지만, 훈련 데이터의 손실이 발생한다는 점을 기억해야 합니다.

#### ✏️ 연습문제 2: 블렌딩 비율 변경하기

  - **문제**: 위 블렌딩 코드에서 `train_test_split`의 `test_size`를 `0.5`로 변경해보세요. 즉, 훈련 데이터의 절반을 기본 모델 학습에, 나머지 절반을 메타 모델 학습에 사용합니다.
  - **목표**: 이 때의 RMSE를 계산하고, `test_size=0.3`일 때와 비교해보세요. `test_size`가 커지면(메타 모델의 학습 데이터가 많아지면) 성능이 어떻게 변하는지, 그 이유는 무엇일지 생각해보세요.

-----

### 5\. 결과 비교 및 결론

지금까지의 베이스라인, 스태킹, 블렌딩 모델의 성능을 한눈에 비교해봅시다.

#### 💻 코드로 알아보기: 최종 성능 비교

In [ ]:
# 결과 DataFrame 생성
results_df = pd.DataFrame.from_dict(baseline_results, orient='index', columns=['RMSE']).sort_values('RMSE')

# 시각화
fig = px.bar(results_df,
             x=results_df.index,
             y='RMSE',
             title='모델별 주택 가격 예측 RMSE 비교',
             labels={'x': '모델', 'RMSE': 'RMSE (낮을수록 좋음)'},
             text_auto='.4f',
             color=results_df.index
            )
fig.show()

print(results_df)

그래프와 표를 통해 어떤 모델의 성능이 가장 좋았는지 명확히 확인할 수 있습니다.

#### 🤔 토의 및 결론

  - **스태킹의 장점**: 이 실습에서는 스태킹 모델이 가장 좋은 성능을 보였을 가능성이 높습니다. 그 이유는 교차 검증을 통해 전체 훈련 데이터를 기본 모델과 메타 모델 학습에 효율적으로 활용했기 때문입니다.  이는 모델들이 데이터의 다양한 패턴을 더 잘 학습하도록 돕습니다.

  - **블렌딩의 장단점**: 블렌딩은 구현이 간단하고 빠르다는 큰 장점이 있습니다.  하지만 훈련 데이터를 분할하기 때문에 기본 모델들이 학습할 데이터가 줄어들어 성능 잠재력을 100% 발휘하기 어려울 수 있습니다.  또한, 메타 모델의 성능이 어떤 홀드아웃 세트가 선택되느냐에 따라 크게 좌우될 수 있다는 불안정성도 있습니다. 

  - **어떤 기법을 언제 사용해야 할까?**

      - **스태킹**: 약간의 성능이라도 더 끌어올리는 것이 중요하고, 컴퓨팅 자원과 시간이 충분할 때 가장 좋은 선택입니다.  특히 캐글과 같은 경진대회에서 상위권을 노릴 때 필수적으로 고려되는 기법입니다. 
      - **블렌딩**: 빠르게 프로토타입을 만들고 싶거나, 구현의 복잡성을 줄이고 싶을 때 유용합니다.  또한, 여러 팀원이 각자 모델을 만들고 그 결과를 간단히 합쳐볼 때 효과적입니다. 